### Purpose of the Notebook
Estimate the reduction in ultimate tensile stress (UTS) $S_u$ and total elongation (TE, or strain at rupture) $\varepsilon_{tr}$ for Tungsten and Tungsten Alloys above the DBTT. Refer to [effect_of_recrystillization.ipynb](../data_exploration/effect_of_recrystillization.ipynb) for initial visual exploration of the data. Define:

$$
\begin{matrix*}[l]
\Delta_{rx} S_u(T) = S_{u,0}(T) - S_{u,rx}(T) \\ 
\Delta_{rx} \varepsilon_{tr}(T) =  \varepsilon_{tr,0}(T) - \varepsilon_{tr,rx}(T) 
\end{matrix*}
$$

Here, $\Delta_{rx} S_u(T),\Delta_{rx} \varepsilon_{tr}(T)$ are the changes in $S_u(T)$ and $\varepsilon_{tr}(T)$ due to recrystallization, with $S_{u,0}(T),\varepsilon_{tr,0}(T) $ the material properties prior to recrystallization and $S_{u,rx}(T),\varepsilon_{tr,rx}(T) $ the material properties of the fully recrystallized material.

There are two items that need to be addressed:

1. Establishing that the reduction(s) are independent of temperature.
2. Estimate the constant reduction.

For (1), we can establish there is no linear dependence using classic hypothesis testing from (simple) linear regression. For (2), the application is conservative, so we'd like to establish a fairly conservative estimate for the maximum reduction/minimum increase in material properties.


In [1]:
from matplotlib import pyplot as plt
import numpy as np
import warnings
from typing import List, Tuple
import sys
sys.path.append('../')
from common_util import NogamiUTSData,ProjectPaths,setup_plotting_format,mk_pdir,markdown_table_from_df

#ignore warnings and set preference for matplotlib styles
warnings.filterwarnings("ignore")
setup_plotting_format()
paths = ProjectPaths()
data = NogamiUTSData()
materials = ['W Plate (H)','K-W Plate (H)','K-W3%Re Plate (H)','K-W3%Re Plate (L)']

dbbt = {'W Plate (H)': 500.0,
        'K-W Plate (H)': 350.0,
        'K-W3%Re Plate (H)': 300.0,
        'K-W3%Re Plate (L)': 300.0}

RT = {'W Plate (H)': 1300.0,
        'K-W Plate (H)': 1300.0,
        'K-W3%Re Plate (H)': 1300.0,
        'K-W3%Re Plate (L)': 1300.0}


### Ultimate Tensile Stress $S_u$ 

#### 1. Linear independence of temperature: Test the hypothesis that $\Delta_{rx} S_u$ is constant  

That is, under the model $\Delta_{rx} S_u = \beta_0 + \beta_1 T + \varepsilon$, with $\varepsilon \sim \mathcal{N}(0,\sigma^2)$ we test the null hypothesis.

$$
\mathcal{H}_0: \hat{\beta}_1 = 0 \qquad \text{vs.} \qquad \mathcal{H}_1: \hat{\beta}_1 \neq 0
$$

The decision rule at confidence level $\alpha$ is:

$$
\begin{matrix*}[l]
\text{Reject} \ \mathcal{H_0} \ \text{in favor of } \mathcal{H_1} \text{ if } |t_0| \geq t_{n-2,1 - \alpha/2} \\
t_0 = \frac{\hat{\beta}_1}{SE(\hat{\beta}_1)} \qquad SE(\hat{\beta}_1) = \sqrt{\frac{S^2}{S_{xx}}}
\end{matrix*}
$$

Where $S^2$ is the unbiased estimate for the variance $\sigma^2$, $S_{xx} = \sum_{i = 1}^n (T_i - \overline{T})$ and $\hat{\beta_1} = S_{xy}/S_{xx}$ is the ML/OLS estimate for $\beta_1$ where $S_{xy} = \sum_{i = 1}^n ((\Delta_{rx} S_u)_i - \overline{\Delta_{rx} S_u})(T_i -\overline{T})$

The below result demonstrates that at $\alpha = 5\%$ we cannot reject $\mathcal{H}_0$.

In [2]:
from scipy.stats.distributions import t as tdist

def test_linear_independence(x: np.ndarray, y: np.ndarray,alpha: float = 0.05) -> float:
    """
    Test for linear independence of the data points
    """
    #calculate the determinant of the matrix
    S_xx = np.sum((x - x.mean())**2)
    S_xy = np.sum((x - x.mean())*(y - y.mean()))
    beta_1 = S_xy/S_xx
    beta_0= y.mean() - beta_1*x.mean()
    y_hat = beta_0 + beta_1*x
    residuals = y - y_hat
    SS_R = np.sum(residuals**2)
    S2 = SS_R/(len(x) - 2)

    #calculate the t-statistic
    t_stat = beta_1/np.sqrt(S2/S_xx)
    t_crit = tdist.ppf(1-alpha/2,len(x)-2)
    return abs(t_stat) < t_crit


for alloy in materials:

    df = data.get_df('UTS [MPa] ' + alloy,keep_column= False)
    rx_df = data.get_df('UTS [MPa] ' + alloy + ' Rx',keep_column= False)
    delta = (df - rx_df)
    delta.dropna(inplace = True)
    delta = delta[delta.index >= dbbt[alloy]]
    delta = delta[delta.index < RT[alloy]]

    H  = test_linear_independence(delta.index.to_numpy(),
                                  delta.to_numpy().squeeze(),
                                  alpha = 0.05)
    print(f'{alloy} is linearly independent?: {H}')

W Plate (H) is linearly independent?: True
K-W Plate (H) is linearly independent?: True
K-W3%Re Plate (H) is linearly independent?: True
K-W3%Re Plate (L) is linearly independent?: True


### 2. Estimation of the Constant Reduction

The mean is not conservative enough. A better estimate would be $\Delta^{max}_{rx} S_u$ where

$$
\mathbb{P}(\Delta_{rx} S_u  < \Delta^{max}_{rx} S_u) = 1 -\alpha
$$

$\Delta_{rx} S_u \sim \mathcal{N}(0, \sigma^2)$ as a consquence of the above analysis, and estimation of the above amounts to computing confidence intervals around of the population. 



In [3]:
import pandas as pd
summary = []
for alloy in materials:

    df = data.get_df('UTS [MPa] ' + alloy,keep_column= False)
    rx_df = data.get_df('UTS [MPa] ' + alloy + ' Rx',keep_column= False)
    delta = (df - rx_df)
    delta.dropna(inplace = True)
    delta = delta[delta.index >= dbbt[alloy]]
    delta = delta[delta.index < RT[alloy]]

    s = float(delta.std())
    dmean = float(delta.mean())
    summary.append((dmean,dmean + s))


summary = pd.DataFrame(np.array(summary),columns = ['Mean [MPa]','Upper Bound [MPa]'],index = materials)
print(np.round(summary))

table = markdown_table_from_df(summary,r'$\Delta_{rx} S_u$','Reduction in UTS due to recrystallization')
with mk_pdir():
    with open(paths.GIT_TABLES.joinpath('UTS_Delta_Summary.tbl'),'w') as f:
        f.write(table)



                   Mean [MPa]  Upper Bound [MPa]
W Plate (H)             272.0              279.0
K-W Plate (H)           252.0              287.0
K-W3%Re Plate (H)       293.0              336.0
K-W3%Re Plate (L)       212.0              291.0


### Total Elongation

#### 1. Linear independence of temperature: Test the hypothesis that $\Delta_{rx} \varepsilon_{tr}$ is constant  

There's not really enough data to conductt this for the K-W3\%Re alloy, so we'll exclude it.

In [4]:
for alloy in materials[:-1]:

    df = data.get_df('TE [%] ' + alloy,keep_column= False)
    rx_df = data.get_df('TE [%] ' + alloy + ' Rx',keep_column= False)
    delta = (df - rx_df)
    delta.dropna(inplace = True)
    delta = delta[delta.index >= dbbt[alloy]]
    delta = delta[delta.index < RT[alloy]]

    H  = test_linear_independence(delta.index.to_numpy(),
                                  delta.to_numpy().squeeze(),
                                  alpha = 0.05)
    print(f'{alloy} is linearly independent?: {H}')

W Plate (H) is linearly independent?: True
K-W Plate (H) is linearly independent?: True
K-W3%Re Plate (H) is linearly independent?: True


### 2. Estimation of the Constant Reduction


In [5]:
import pandas as pd
summary = []
for alloy in materials[:-1]:

    df = data.get_df('TE [%] ' + alloy,keep_column= False)
    rx_df = data.get_df('TE [%] ' + alloy + ' Rx',keep_column= False)
    delta = (df - rx_df)
    delta.dropna(inplace = True)
    delta = delta[delta.index >= dbbt[alloy]]
    delta = delta[delta.index < RT[alloy]]

    s = float(delta.std())
    dmean = float(delta.mean())
    summary.append((dmean,dmean + s))


summary = pd.DataFrame(np.array(summary),columns = ['Mean [%]','Lower Bound [%]'],index = materials[:-1])

table = markdown_table_from_df(summary,r'$\Delta_{rx} \varepsilon{tr}$','Reduction in TE due to recrystallization')
with mk_pdir():
    with open(paths.GIT_TABLES.joinpath('TE_Delta_Summary.tbl'),'w') as f:
        f.write(table)